# Twitter API

**Rules:**
- Request window is 15 minute.
- Requests per 15 minutes is as little as 15 requests, and as much as 900 requests.
- See Twitter rate limit documentation here: https://developer.twitter.com/en/docs/basics/rate-limiting
- See Twitter Basic API endpoint documentation here: https://developer.twitter.com/en/docs/basics/rate-limits
- See complete Twitter API Reference Index here: https://developer.twitter.com/en/docs/api-reference-index

### Twitter GET Endpoints

- **/1.1/search/tweets.json**
    - Returns a collection of relevant Tweets matching a specified query. The Twitter Search API searches against a sampling of recent Tweets published in the past 7 days. It’s important to know that the standard search API is focused on relevance and not completeness.
        - id (of the tweet)
        - id (of the user)
        - name (of the user)
        - id (user_mentions)
        - name (user_mentions)
        - text (of the tweet)
        - location
        - entities
        - followers_count (user)
        - friends_count (user)
        - listed_count 
        - favourites_count
        - statuses_count
        - retweet_count (of the tweet)
        - favorite_count
        
        
- **/1.1/trends/place.json**
    - Returns the top 50 trending topics for a specific WOEID, if trending information is available for it. The tweet_volume for the last 24 hours is also returned for many trends if this is available.
        - name (of the trend)
        - tweet_volume


### Twitter Labs GET Endpoints


- **/labs/1/users**
    - Returns a variety of information about one or more Users specified by the requested IDs or usernames.
        - description
        - id
        - location
        - name
        - username
        - stats
            - followers_count
            - following_count
            - tweet_count
            - listed_count
            
            
- **/labs/1/tweets**
    - Returns a variety of information about the Tweet specified by the requested ID or list of IDs.
        - attachments (to the tweet)
        - stats (for the tweet)
            - retweet_count
            - reply_count
            - like_count
            - quote_count
        - text (tweet content)
        
        
- **/labs/1/tweets/search**
    - Returns Tweets from the last 7 days that match a search query (max 100 results per call).
        - all keys from /search/tweets.json
        - next_token (to paginate)
        
        
- **/labs/1/tweets/stream/filter**
    - Streams Tweets in real-time based on a specific set of filter rules.
        - all keys from /search/tweets.json
        - next_token (to paginate)
        
   
- **/labs/1/tweets/stream/sample**
    - Streams about 1% of all Tweets in real-time. (TAKES A LONG TIME: >10 MINUTES)
        - ???
        - ???
        - ???

### Workflow
- **Step 1.** Use **/1.1/search/tweets.json** endpoint to get tweets from the past 7 days for a particular query. 

- **Step 2.** Use **/labs/1/tweets** to get detailed tweet metrics using the tweet ID extracted from Step 1.

- **Step 3.** Use **/labs/1/users** to get detailed info about the users for the tweets in our query from Step 1.

- **Step 4.** Merge all data into one DataFrame and export to .CSV.

___

# Use Cases

- Classify if a **tweet** will cross a **high threshold of likes** using certain predictors (hashtags, followers, total user tweets, retweets, reply count, quote count, location, etc...)


- Classify if a **user** will cross a **high threshold of followers** using certain predictors (hashtags, likes, total user tweets, retweets, reply count, quote count, location, etc...)


- Classify if a **tweet** will cross a **high threshold of retweets** using certain predictors (hashtags, likes, total user tweets, followers, reply count, quote count, location, etc...)

___

# Business Cases

- A company wants to know how to get more likes on their tweet to get more people interested in their products and services.


- A wannabe influencer wants to know how to get a high threshold of followers to get more companies to invest in him/her to promote products and services.


- A startup company just deployed a new app and wants to get the word out. They want to maximize retweets of their "launch" tweet about their new app. They want to know how to make a tweet cross a high threshold of retweets.

___

# Setup

In [1]:
import requests
import json

In [4]:
client_key = 'DaaVvjhdnfMrmU7cJcciQQuil'
client_secret = 'etyYWRFf6wwyYSJvjyr2JxdcxHzlzfnsPcBdOq68GY9qwVf6RH'

import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [5]:
base_url = 'https://api.twitter.com/'
auth_url = '{}oauth2/token'.format(base_url)

auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

auth_data = {
    'grant_type': 'client_credentials'
}

auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

In [6]:
auth_resp.status_code

200

In [7]:
auth_resp.json().keys()

dict_keys(['token_type', 'access_token'])

In [8]:
auth_resp.json()

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz'}

___

# Test Call To /1.1/search/tweets.json Endpoint

**Returns a collection of relevant Tweets matching a specified query.**

In [9]:
search_headers = {
    'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
}

search_params = {
    'q': 'General Election',
    'result_type': 'recent',
    'count': 2
}

search_url = '{}1.1/search/tweets.json'.format(base_url)

search_resp = requests.get(search_url, headers=search_headers, params=search_params)
response = search_resp.json()
response

{'statuses': [{'created_at': 'Fri Jan 31 00:22:27 +0000 2020',
   'id': 1223038801519562752,
   'id_str': '1223038801519562752',
   'text': "RT @jvgraz: While centrists clutch their pearls over Bernie's fictional oppo research file, just imagine this video played on a loop during…",
   'truncated': False,
   'entities': {'hashtags': [],
    'symbols': [],
    'user_mentions': [{'screen_name': 'jvgraz',
      'name': "Plain Ol' Johnny Graz",
      'id': 16959075,
      'id_str': '16959075',
      'indices': [3, 10]}],
    'urls': []},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_id_str': None,
   'in_reply_to_user_id': None,
   'in_reply_to_user_id_str': None,
   'in_reply_to_screen_name': None,
   'user': {'id': 975373644959449089,
    'id_str': '975373644959449089',
    'name': 'theMAOrganic',
    'screen_na

In [10]:
# with open('search_tweets.json', 'w', encoding='utf-8') as f:
#     json.dump(response, f, ensure_ascii=False, indent=4)

In [24]:
for x in response['statuses']:
    print(x['text'] + '\n')

RT @peterdaou: FACT CHECK: IS #BERNIE VETTED?

I've forcefully rebutted the myth that @BernieSanders faces a damaging oppo dump in a genera…

RT @damian_from: Starmer and Thornberry got it terribly wrong.

Their refusal to respect the largest mandate in UK history (17.4m leave vot…



In [25]:
#two primary keys in the response
response.keys()

dict_keys(['statuses', 'search_metadata'])

### "statuses"

In [29]:
#what keys are in the "statuses" dictionary?
list(response['statuses'][0].keys())

['created_at',
 'id',
 'id_str',
 'text',
 'truncated',
 'entities',
 'metadata',
 'source',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'user',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'retweeted_status',
 'is_quote_status',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'lang']

In [37]:
#looks like the text of the tweet is cut off after a certain number of characters.
#there is also some clean up involved with "\n" inserted instead of spaces.

response['statuses'][0]['text'].replace('\n',' ')

"RT @peterdaou: FACT CHECK: IS #BERNIE VETTED?  I've forcefully rebutted the myth that @BernieSanders faces a damaging oppo dump in a genera…"

In [47]:
#"entities" shows who the tweet "@" mentioned, and which "#" hashtags they used in the tweet.
response['statuses'][0]['entities']

{'hashtags': [{'text': 'BERNIE', 'indices': [30, 37]}],
 'symbols': [],
 'user_mentions': [{'screen_name': 'peterdaou',
   'name': 'Peter Daou',
   'id': 18464266,
   'id_str': '18464266',
   'indices': [3, 13]},
  {'screen_name': 'BernieSanders',
   'name': 'Bernie Sanders',
   'id': 216776631,
   'id_str': '216776631',
   'indices': [86, 100]}],
 'urls': []}

In [38]:
#what's in "metadata"?
response['statuses'][0]['metadata']

{'iso_language_code': 'en', 'result_type': 'recent'}

In [39]:
#"user" key has a LOT of juicy information about the Twitter user.
#name, followers, friends, date Twitter account was made, total number of "statuses", profile image, etc...
response['statuses'][0]['user']

{'id': 773966501451890688,
 'id_str': '773966501451890688',
 'name': 'Pat Miguel Tomaino',
 'screen_name': 'Pat_Tomaino',
 'location': 'Boston',
 'description': 'Research & activism at @ZevinAssetMgmt. Mexican American socialist he/him. Board/adviser: @UUA, @UjimaBoston. Former researcher: Warren 2012, @1199SEIU 🌹🇲🇽\U0001f9a7',
 'url': None,
 'entities': {'description': {'urls': []}},
 'protected': False,
 'followers_count': 1325,
 'friends_count': 5001,
 'listed_count': 25,
 'created_at': 'Thu Sep 08 19:29:28 +0000 2016',
 'favourites_count': 20103,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': False,
 'statuses_count': 14230,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'is_translation_enabled': False,
 'profile_background_color': '000000',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_ba

In [43]:
#source of tweet (iPhone, Android, etc...)
response['statuses'][0]['source']

'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'

In [ ]:
#note that the following keys have no data...Twitter probably restricts this info.
# 'geo'
# 'coordinates'
# 'place'
# 'contributors'

In [48]:
response['statuses'][0]['retweeted_status']

{'created_at': 'Wed Jan 29 14:19:35 +0000 2020',
 'id': 1222524697117646849,
 'id_str': '1222524697117646849',
 'text': "FACT CHECK: IS #BERNIE VETTED?\n\nI've forcefully rebutted the myth that @BernieSanders faces a damaging oppo dump in… https://t.co/mjiV2VOJiO",
 'truncated': True,
 'entities': {'hashtags': [{'text': 'BERNIE', 'indices': [15, 22]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'BernieSanders',
    'name': 'Bernie Sanders',
    'id': 216776631,
    'id_str': '216776631',
    'indices': [71, 85]}],
  'urls': [{'url': 'https://t.co/mjiV2VOJiO',
    'expanded_url': 'https://twitter.com/i/web/status/1222524697117646849',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [117, 140]}]},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_use

In [49]:
response['statuses'][0]['retweet_count']

661

In [50]:
response['statuses'][0]['favorite_count']

0

In [51]:
response['statuses'][0]['retweeted']

False

In [52]:
response['statuses'][0]['lang']

'en'

___

### "search_metadata"

In [30]:
#what keys are in the "search_metadata" dictionary?
list(response['search_metadata'].keys())

['completed_in',
 'max_id',
 'max_id_str',
 'next_results',
 'query',
 'refresh_url',
 'count',
 'since_id',
 'since_id_str']

In [53]:
response['search_metadata']

{'completed_in': 0.022,
 'max_id': 1222603464804589569,
 'max_id_str': '1222603464804589569',
 'next_results': '?max_id=1222603459377205247&q=General%20Election&count=2&include_entities=1&result_type=recent',
 'query': 'General+Election',
 'refresh_url': '?since_id=1222603464804589569&q=General%20Election&result_type=recent&include_entities=1',
 'count': 2,
 'since_id': 0,
 'since_id_str': '0'}

# Test Call To /1.1/trends/place.json Endpoint

**Returns the top 50 trending topics for a specific WOEID, if trending information is available for it. The tweet_volume for the last 24 hours is also returned for many trends if this is available.**

The response is an array of trend objects that encode the name of the trending topic, the query parameter that can be used to search for the topic on Twitter Search, and the Twitter Search URL.

In [90]:
search_headers = {
    'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
}

search_params = {
    'id': '1'
}

search_url = 'https://api.twitter.com/1.1/trends/place.json'

request_0 = requests.get(search_url, headers=search_headers, params=search_params)
trends = request_0.json()
trends

[{'trends': [{'name': '#BBBB20',
    'url': 'http://twitter.com/search?q=%23BBBB20',
    'promoted_content': None,
    'query': '%23BBBB20',
    'tweet_volume': 73478},
   {'name': '#AEWDynamite',
    'url': 'http://twitter.com/search?q=%23AEWDynamite',
    'promoted_content': None,
    'query': '%23AEWDynamite',
    'tweet_volume': 19901},
   {'name': 'Marcela',
    'url': 'http://twitter.com/search?q=Marcela',
    'promoted_content': None,
    'query': 'Marcela',
    'tweet_volume': 281772},
   {'name': '#BellLetsTaIk',
    'url': 'http://twitter.com/search?q=%23BellLetsTaIk',
    'promoted_content': None,
    'query': '%23BellLetsTaIk',
    'tweet_volume': 21218},
   {'name': '#sbgcbud',
    'url': 'http://twitter.com/search?q=%23sbgcbud',
    'promoted_content': None,
    'query': '%23sbgcbud',
    'tweet_volume': 12872},
   {'name': '#RokuSurroundSound',
    'url': 'http://twitter.com/search?q=%23RokuSurroundSound',
    'promoted_content': None,
    'query': '%23RokuSurroundSound'

In [95]:
len(trends[0]['trends'])

50

In [98]:
#let's see tweet_volumes

volume=[]
for x in trends[0]['trends']:
    volume.append(x['tweet_volume'])
volume

[73478,
 19901,
 281772,
 21218,
 12872,
 None,
 64751,
 None,
 None,
 16467,
 125490,
 None,
 33844,
 13127,
 None,
 None,
 None,
 None,
 None,
 16164,
 None,
 10568,
 52126,
 None,
 153874,
 15193,
 None,
 None,
 78336,
 None,
 22985,
 None,
 67334,
 None,
 None,
 None,
 29022,
 None,
 None,
 None,
 None,
 28274,
 15777,
 324037,
 None,
 None,
 32530,
 None,
 None,
 None]

___

# Twitter Labs Endpoints

# Test Call To "labs/1/users" Endpoint

**Getting information about a particular Twitter user here (Donald Trump)**

In [13]:
search_headers = {
    'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
}

search_params = {
    'usernames': 'realDonaldTrump,Twitter,MichelleObama,sundarpichai,tim_cook',
    'user.format': 'detailed',
    'format': 'detailed'
}

search_url = 'https://api.twitter.com/labs/1/users?'

request_1 = requests.get(search_url, headers=search_headers, params=search_params)
users = request_1.json()
users

{'data': [{'created_at': '2009-03-18T13:46:38.000Z',
   'description': '45th President of the United States of America🇺🇸',
   'entities': {'url': {'urls': [{'start': 0,
       'end': 23,
       'url': 'https://t.co/OMxB0x7xC5',
       'expanded_url': 'http://www.Instagram.com/realDonaldTrump',
       'display_url': 'Instagram.com/realDonaldTrump'}]}},
   'id': '25073877',
   'location': 'Washington, DC',
   'most_recent_tweet_id': '1223038027234267137',
   'name': 'Donald J. Trump',
   'pinned_tweet_id': '1222655839573565440',
   'profile_image_url': 'https://pbs.twimg.com/profile_images/874276197357596672/kUuht00m_normal.jpg',
   'protected': False,
   'stats': {'followers_count': 71873110,
    'following_count': 47,
    'tweet_count': 48500,
    'listed_count': 113961},
   'url': 'https://t.co/OMxB0x7xC5',
   'username': 'realDonaldTrump',
   'verified': True,
   'format': 'detailed'},
  {'created_at': '2007-02-20T14:35:54.000Z',
   'description': 'What’s happening?!',
   'entities':

In [14]:
# with open('users.json', 'w', encoding='utf-8') as f:
#     json.dump(users, f, ensure_ascii=False, indent=4)

# Test Call To "labs/1/tweets" Endpoint

**Getting detailed information about a tweet here (most recent tweet from Donald Trump)**

In [15]:
search_headers = {
    'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
}

search_params = {
    'ids': '1223038027234267137,1222972807639896064,1219257438949597185,1221298825786089472,1221813908576464896',
    'format': 'detailed',
    'tweet.format': 'detailed',
    'user.format': 'detailed',
    'place.format': 'detailed'
}

search_url = 'https://api.twitter.com/labs/1/tweets'

request_2 = requests.get(search_url, headers=search_headers, params=search_params)
get_tweets = request_2.json()
get_tweets

{'data': [{'attachments': {'media_keys': ['3_1223038022867935232']},
   'author_id': '25073877',
   'created_at': '2020-01-31T00:19:23.000Z',
   'entities': {'hashtags': [{'start': 57, 'end': 65, 'tag': 'KAG2020'}],
    'urls': [{'start': 68,
      'end': 91,
      'url': 'https://t.co/4YCo01XYCn',
      'expanded_url': 'https://twitter.com/realDonaldTrump/status/1223038027234267137/photo/1',
      'display_url': 'pic.twitter.com/4YCo01XYCn'}]},
   'id': '1223038027234267137',
   'lang': 'en',
   'possibly_sensitive': False,
   'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
   'stats': {'retweet_count': 2406,
    'reply_count': 1346,
    'like_count': 8517,
    'quote_count': 192},
   'text': 'Great poll in Iowa, where I just landed for a Big Rally! #KAG2020🇺🇸 https://t.co/4YCo01XYCn',
   'format': 'detailed'},
  {'author_id': '783214',
   'created_at': '2020-01-30T20:00:13.000Z',
   'id': '1222972807639896064',
   'lang': 'en',
   'poss

In [16]:
# with open('tweets_deets.json', 'w', encoding='utf-8') as f:
#     json.dump(get_tweets, f, ensure_ascii=False, indent=4)

# Test Call To "/labs/1/tweets/search" Endpoint

**/labs/1/tweets/search endpoint returns Tweets from the last 7 days that match a search query. Query is for "Kobe".**

In [76]:
search_headers = {
    'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
}

search_params = {
    'query': 'kobe',
    'max_results': 100,
    'format': 'detailed',
    'tweet.format': 'detailed',
    'user.format': 'detailed',
    'place.format': 'detailed'
}

search_url = 'https://api.twitter.com/labs/1/tweets/search'

request_3 = requests.get(search_url, headers=search_headers, params=search_params)
tweets_search = request_3.json()
tweets_search

{'data': [{'attachments': {'media_keys': ['13_1222613410485014529']},
   'author_id': '2742356057',
   'created_at': '2020-01-30T00:40:35.000Z',
   'entities': {'mentions': [{'start': 3, 'end': 8, 'username': 'espn'}],
    'urls': [{'start': 74,
      'end': 97,
      'url': 'https://t.co/ThBIjzKWVN',
      'expanded_url': 'https://twitter.com/espn/status/1222614701277949952/video/1',
      'display_url': 'pic.twitter.com/ThBIjzKWVN'}]},
   'id': '1222680977165647874',
   'lang': 'en',
   'possibly_sensitive': False,
   'referenced_tweets': [{'type': 'retweeted', 'id': '1222614701277949952'}],
   'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
   'stats': {'retweet_count': 4970,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': 'RT @espn: Zaire Wade hit the Kobe celebration after throwing down a dunk. https://t.co/ThBIjzKWVN',
   'format': 'detailed'},
  {'author_id': '1113108537239048194',
   'created_at': '202

In [79]:
tweets_search['data'][0]

{'attachments': {'media_keys': ['13_1222613410485014529']},
 'author_id': '2742356057',
 'created_at': '2020-01-30T00:40:35.000Z',
 'entities': {'mentions': [{'start': 3, 'end': 8, 'username': 'espn'}],
  'urls': [{'start': 74,
    'end': 97,
    'url': 'https://t.co/ThBIjzKWVN',
    'expanded_url': 'https://twitter.com/espn/status/1222614701277949952/video/1',
    'display_url': 'pic.twitter.com/ThBIjzKWVN'}]},
 'id': '1222680977165647874',
 'lang': 'en',
 'possibly_sensitive': False,
 'referenced_tweets': [{'type': 'retweeted', 'id': '1222614701277949952'}],
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'stats': {'retweet_count': 4970,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'text': 'RT @espn: Zaire Wade hit the Kobe celebration after throwing down a dunk. https://t.co/ThBIjzKWVN',
 'format': 'detailed'}

# Test Call To "/labs/1/tweets/stream/search" Endpoint

**The Labs recent search endpoint returns Tweets from the last 7 days that match a search query.**

In [82]:
search_headers = {
    'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
}

search_params = {
    'query': 'data',
    'format': 'detailed',
    'tweet.format': 'detailed',
    'user.format': 'detailed',
    'place.format': 'detailed'
}

search_url = 'https://api.twitter.com/labs/1/tweets/search'

request_4 = requests.get(search_url, headers=search_headers, params=search_params)
tweets_stream = request_4.json()
tweets_stream

{'data': [{'attachments': {'media_keys': ['7_1222670186265960448']},
   'author_id': '735610146001424385',
   'created_at': '2020-01-30T00:47:13.000Z',
   'entities': {'hashtags': [{'start': 105, 'end': 114, 'tag': 'PS4share'}],
    'mentions': [{'start': 3, 'end': 12, 'username': 'acerikus'}],
    'urls': [{'start': 115,
      'end': 138,
      'url': 'https://t.co/Pd5iXup0nl',
      'expanded_url': 'https://twitter.com/acerikus/status/1222670895833133060/video/1',
      'display_url': 'pic.twitter.com/Pd5iXup0nl'}]},
   'id': '1222682645844369410',
   'lang': 'en',
   'possibly_sensitive': False,
   'referenced_tweets': [{'type': 'retweeted', 'id': '1222670895833133060'}],
   'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
   'stats': {'retweet_count': 10,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': "RT @acerikus: Here's  a quick guide to using both sides of the island at the same time in data greeting! #PS4share h

In [87]:
#10 real-time streamed tweets are returned from around the world for the query
#you can use 'next_token' to paginate through Twitter results
tweets_stream['meta']['result_count']

10

# Test Call To "/labs/1/tweets/stream/sample" Endpoint

**/labs/1/tweets/stream/sample endpoint Streams about 1% of all Tweets in real-time.**

In [ ]:
#code commented out because it takes too long to run.

# search_headers = {
#     'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
# }

# search_params = {
#     'format': 'detailed',
#     'tweet.format': 'detailed',
#     'user.format': 'detailed',
#     'place.format': 'detailed'
# }

# search_url = 'https://api.twitter.com/labs/1/tweets/stream/sample'

# request_5 = requests.get(search_url, headers=search_headers, params=search_params)
# tweets_sample = request_5.json()
# tweets_sample

# Test Call To "/labs/1/tweets/stream/filter" Endpoint

**/labs/1/tweets/stream/filter endpoint streams Tweets in real-time based on a specific set of filter rules**

In [82]:
search_headers = {
    'Authorization': 'Bearer {}'.format('AAAAAAAAAAAAAAAAAAAAAAvlCAEAAAAAwtmjoshI4rUavHrUhCkw%2F6lBezc%3DP8G7CwGZUMYAfcgjcrX8zuRcGCzEn7VDudpctRjNMPRMo762cz')    
}

search_params = {
    'query': 'data',
    'format': 'detailed',
    'tweet.format': 'detailed',
    'user.format': 'detailed',
    'place.format': 'detailed'
}

search_url = 'https://api.twitter.com/labs/1/tweets/search'

request_4 = requests.get(search_url, headers=search_headers, params=search_params)
tweets_stream = request_4.json()
tweets_stream

{'data': [{'attachments': {'media_keys': ['7_1222670186265960448']},
   'author_id': '735610146001424385',
   'created_at': '2020-01-30T00:47:13.000Z',
   'entities': {'hashtags': [{'start': 105, 'end': 114, 'tag': 'PS4share'}],
    'mentions': [{'start': 3, 'end': 12, 'username': 'acerikus'}],
    'urls': [{'start': 115,
      'end': 138,
      'url': 'https://t.co/Pd5iXup0nl',
      'expanded_url': 'https://twitter.com/acerikus/status/1222670895833133060/video/1',
      'display_url': 'pic.twitter.com/Pd5iXup0nl'}]},
   'id': '1222682645844369410',
   'lang': 'en',
   'possibly_sensitive': False,
   'referenced_tweets': [{'type': 'retweeted', 'id': '1222670895833133060'}],
   'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
   'stats': {'retweet_count': 10,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': "RT @acerikus: Here's  a quick guide to using both sides of the island at the same time in data greeting! #PS4share h

In [104]:
tweets_stream['meta']['result_count']

10